In [1]:
!pip install clifford

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 15.4 MB/s eta 0:00:00


In [2]:
# @title A.1 Setup and Algebra Construction
import clifford
import numpy as np

T = complex(0,1)
p,q = 5,2  # any signature that p+q=7 is valid
layout, blades = clifford.Cl(p, q)
e = blades['1'] = blades['e1']*blades['e1'] *blades['e1'] *blades['e1']
del blades['']


p_, q_ = (0,q-1) if p==0 else (q,p-1)
layout_, blades_ = clifford.Cl(p_, q_)
E = blades_['1'] = blades_['e1']*blades_['e1'] *blades_['e1'] *blades_['e1']
blades_ = {key.upper(): value for (key,value) in blades_.items()}
del blades_['']

locals().update(blades)
BASIS_CONJ = {
    0: [1, e1234567],
    1: [e12, e13, e14, e15, e16, e17, e34567, e24567, e23567, e23467, e23457, e23456],
    2: [e23, e24, e25, e26, e27, e34, e35, e36, e37, e45, e46, e47, e56, e57, e67,
        e14567, e13567, e13467, e13457, e13456, e12567, e12467, e12457, e12456,
        e12367, e12357, e12356, e12347, e12346, e12345],
    3: [e1234, e1235, e1236, e1237, e1245, e1246, e1247, e1256, e1257, e1267, e1345,
        e1346, e1347, e1356, e1357, e1367, e1456, e1457, e1467, e1567, e567, e467,
        e457, e456, e367, e357, e356, e347, e346, e345, e267, e257, e256, e247, e246,
        e245, e237, e236, e235, e234],
    4: [e2345, e2346, e2356, e2456, e3456, e2347, e2357, e2457, e3457, e2367, e2467,
        e3467, e2567, e3567, e4567, e167, e157, e147, e137, e127, e156, e146, e136,
        e126, e145, e135, e125, e134, e124, e123],
    5: [e2, e3, e4, e5, e6, e7, e134567, e124567, e123567, e123467, e123457, e123456],
    6: [e1, e234567],
}

locals().update(blades_)

generators  = [e1,e2,e3,e4,e5,e6,e7]
generators_ = [E1,E2,E3,E4,E5,E6]


def b(el, overline_ids):
    for id in overline_ids:
        el = el - 2 * sum(el[i] * i for i in BASIS_CONJ[id])
    return el

U_BASE = sum(blades.values())
U_BASE_ = sum(blades_.values())

b_4_signs   = b(U_BASE, [4]).value
b_145_signs = b(U_BASE,  [1, 4, 5]).value
b_236_signs = b(U_BASE,  [2, 3, 6]).value

m_4_signs      = (U_BASE - 2 * U_BASE(4)).value
m_4_signs_     = (U_BASE_ - 2 * U_BASE_(4)).value
m_34_signs     = (U_BASE - 2 * U_BASE(3, 4)).value
m_145_signs_   = (U_BASE_ - 2 * U_BASE_(1,4,5)).value
m_145_signs    = (U_BASE - 2 * U_BASE(1,4,5)).value
m_123456_signs = (U_BASE - 2 * U_BASE(1, 2, 3, 4, 5, 6)).value


b_4      = lambda el: layout.MultiVector(el.value*b_4_signs)
b_145    = lambda el: layout.MultiVector(el.value*b_145_signs)
b_236    = lambda el: layout.MultiVector(el.value*b_236_signs)

m_4      = lambda el: layout.MultiVector(el.value*m_4_signs)
m_4_     = lambda el: layout_.MultiVector(el.value*m_4_signs_)
m_34     = lambda el: layout.MultiVector(el.value*m_34_signs)
m_145_   = lambda el: layout_.MultiVector(el.value*m_145_signs_)
m_145    = lambda el: layout.MultiVector(el.value*m_145_signs)
m_123456 = lambda el: layout.MultiVector(el.value*m_123456_signs)

In [3]:
# @title A.1 Setup phi isomorphism

from itertools import combinations
from itertools import product
import numpy as np
import math


def powerset(input_list):
    subsets = []
    for i in range(len(input_list) + 1):
        for combo in combinations(input_list, i):
            subsets.append(list(combo))
    return subsets

def gen_phi(generators,generators_lower=[]):
    '''

    '''
    E_i = {str(i):generators[0]*gen for i,gen in enumerate(generators[1:],1)}
    E_c = math.prod(generators)
    indexes = powerset(E_i.keys())

    E_real    = {f'1*E{''.join(i)}': math.prod(E_i[j] for j in i) for i in indexes}
    E_complex = {'T'+i[1:]: E_c*E_real[i] for i in E_real}
    E_real.update(E_complex)
    E = E_real
    return E

def gen_phi_inv(E_):
    E_inv = dict()
    for e_lower, e  in E_.items():
        sign = 1 if e==1 else e.value.sum()
        value = ('-' if (sign<0) else '+')+e_lower
        E_inv[(str(sign*e))] = eval(value)
    return E_inv

def find_ind(arr):
    if arr == 1 or arr == 1j :
        return 0
    return np.where((arr.value.__abs__()>0))[0][0]
phi_maping = gen_phi(generators)
phi_maping['1*E'] = e
inv_phi_mapping = gen_phi_inv(phi_maping)

toE6_complex    = np.array([find_ind(E7_) for (E6_,E7_) in phi_maping.items() if eval(E6_).value.sum().imag!=0])
toE6_real       = np.array([find_ind(E7_) for (E6_,E7_) in phi_maping.items() if eval(E6_).value.sum().imag==0])
toE7            = np.empty(len(phi_maping),dtype=int)
is_real_mask    = np.empty(len(phi_maping),dtype=bool)

for i6,i7 in phi_maping.items():
    toE7[int(np.where(i7.value)[0][0])]= int(np.where(eval(i6).value)[0][0])
    is_real_mask[int(np.where(i7.value)[0][0])]= eval(i6).value.sum().imag==0

def phi(u7):
    u6 = layout_.MultiVector()
    u6.value = u7.value[toE6_real]+T*u7.value[toE6_complex]
    return u6

def phi_inv(u6):
    u7 = layout.MultiVector()
    u7.value = u6.value.real[toE7]*is_real_mask+u6.value.imag[toE7]*(~is_real_mask)
    return u7

In [4]:
# @title A.2  Hitzer & Sangwine Formula for Determinant and Inverse for $n=7$

# m_145_ = lambda el: el-2*sum(el(i) for i in [1,4,5])
# m_4_ = lambda el: el-2*sum(el(i) for i in [4])
def cl6_adj(u):
    u_456 = (~u)
    w = u * u_456
    temp = m_4_(w)
    term1 = w * m_145_(w * w)
    term2 = 2 * m_4_(temp * m_145_(temp * temp))
    y = -u_456*(term1 + term2) / 3
    return y

def cl7_determinant_hitzersangwine(u,phi=phi,phi_inv=phi_inv,eps=1e-7):
    b = phi(u)
    y = cl6_adj(b)
    p_inv = phi_inv(b*y)
    p_inv.value[np.abs(p_inv.value) < eps] = 0
    return p_inv*(p_inv.gradeInvol())

def cl7_inverse_hitzersangwine(u,phi=phi,phi_inv=phi_inv,eps=1e-7):
    b = phi(u)
    u_456 = (~b)
    w = b * u_456
    temp = m_4_(w)
    term1 = w * m_145_(w * w)
    term2 = 2 * m_4_(temp * m_145_(temp * temp))
    y = u_456*(term1 + term2) / 3
    y_inv = phi_inv(y)
    by_inv = phi_inv(b*y)
    by_inv.value[np.abs(by_inv.value) < eps] = 0
    determinant = -(by_inv * by_inv.gradeInvol()).value[0]
    if abs(float(determinant)) < 1e-12:
        raise ZeroDivisionError("Determinant is zero or near zero — inverse does not exist.")
    return (y_inv * by_inv.gradeInvol()) /determinant

In [5]:
# @title A.2 Explicit Formula for Determinant and Inverse for $n=7$
def cl7_determinant(u):
    u_236 = b_236(u)
    w = u * u_236
    temp = b_4(w)
    term1 = w * b_145(w * w)
    term2 = 2 * b_4(temp * b_145(temp * temp))
    uy = w*(term1 + term2) / 3
    return (uy * uy.gradeInvol())

def cl7_inverse(u):
    u_236 = b_236(u)
    w = u * u_236
    temp = b_4(w)
    term1 = w * b_145(w * w)
    term2 = 2 * b_4(temp * b_145(temp * temp))
    y = u_236 * (term1 + term2) / 3
    uy = u * y
    determinant = (uy * uy.gradeInvol()).value[0]
    if abs(float(determinant)) < 1e-12:
        raise ZeroDivisionError("Determinant is zero or near zero — inverse does not exist.")
    return (y * uy.gradeInvol()) / determinant

In [6]:
# @title A.3 Optimized Explicit Formula for Determinant and Inverse for $n=7$
def cl7_determinant_opt(u):
    u_236 = b_236(u)
    w = u * u_236
    temp = m_34(w)
    term1 = w * m_123456(w * w)
    term2 = 2 * m_34(temp * m_123456(temp * temp))
    uy = w*(term1 + term2) / 3
    return (uy * uy.gradeInvol())

def cl7_inverse_opt(u):
    u_236 = b_236(u)
    w = u * u_236
    temp = m_34(w)
    term1 = w * m_123456(w * w)
    term2 = 2 * m_34(temp * m_123456(temp * temp))
    y = u_236 * (term1 + term2) / 3
    uy = u * y
    determinant = (uy * uy.gradeInvol()).value[0]
    if abs(float(determinant)) < 1e-12:
        raise ZeroDivisionError("Determinant is zero or near zero — inverse does not exist.")
    return (y * uy.gradeInvol()) / determinant

In [7]:
# @title B.1 Unit Tests for the Implementation
import unittest
from numpy.linalg import norm
import numpy as np

class TestCl7Algebra(unittest.TestCase):
    def setUp(self):
        # Basis blades and random multivectors
        self.layout = layout
        self.blades = blades
        self.e = [blades[f"e{i+1}"] for i in range(7)]
        np.random.seed(42)  # for reproducibility

    def rand_multivector(self):
        return sum(np.random.randn() * blade/10 for blade in self.layout.blades_list)

    def test_determinant_is_scalar(self):
        """Determinants must be of grade 0 (i.e., scalars)."""
        for _ in range(10):
            u = self.rand_multivector()
            det_hit = cl7_determinant_hitzersangwine(u)
            det_std = cl7_determinant(u)
            det_opt = cl7_determinant_opt(u)

            self.assertEqual(det_hit.grades(), {0}, "Hitzer determinant is not scalar")
            self.assertEqual(det_std.grades(), {0}, "Standard determinant is not scalar")
            self.assertEqual(det_opt.grades(), {0}, "Optimized determinant is not scalar")

    def test_inverse_consistency(self):
        """Inverses from both implementations should match."""
        for _ in range(10):
            u = 2*self.rand_multivector()
            try:
                inv1 = cl7_inverse(u)
                inv2 = cl7_inverse_opt(u)
                diff =  norm((inv1 - inv2).value)
                self.assertAlmostEqual(diff, 0, places=10)
            except ZeroDivisionError:
                pass  # skip singular cases

    def test_inverse_correctness(self):
        """Check that u * u^{-1} is close to 1."""
        for _ in range(10):
            u = self.rand_multivector()
            try:
                inv = cl7_inverse(u)
                result = (u * inv).normal()
                self.assertAlmostEqual(result[()], 1.0, places=6)
            except ZeroDivisionError:
                pass  # skip non-invertible

    def test_inverse_opt_correctness(self):
        """Check that u * u_opt^{-1} is close to 1."""
        for _ in range(10):
            u = self.rand_multivector()
            try:
                inv = cl7_inverse_opt(u)
                result = norm((u * inv).value)
                self.assertAlmostEqual(result[()], 1.0, places=6)
            except ZeroDivisionError:
                pass

    def test_inverse_hitzer_correctness(self):
        """Check that u * u_hitzer^{-1} is close to 1."""
        for _ in range(10):
            u = self.rand_multivector()
            try:
                inv = cl7_inverse_hitzersangwine(u)
                result = norm((u * inv).value)
                self.assertAlmostEqual(result[()], 1.0, places=6)
            except ZeroDivisionError:
                pass

    def test_singular_detection(self):
        """Determinant near-zero should raise ZeroDivisionError in inverse functions."""
        u = 0 * e1  # clearly singular
        with self.assertRaises(ZeroDivisionError):
            cl7_inverse(u)
        with self.assertRaises(ZeroDivisionError):
            cl7_inverse_opt(u)

    def test_idempotent_behavior(self):
        """Check that inverse(inverse(u)) is close to  u."""
        for _ in range(5):
            u = self.rand_multivector()
            try:
                inv = cl7_inverse(u)
                inv_inv = cl7_inverse(inv)
                self.assertLess(norm((u - inv_inv).value), 1e2)
            except ZeroDivisionError:
                pass

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....F..
FAIL: test_inverse_hitzer_correctness (__main__.TestCl7Algebra.test_inverse_hitzer_correctness)
Check that u * u_hitzer^{-1} is close to 1.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-2924433469.py", line 70, in test_inverse_hitzer_correctness
    self.assertAlmostEqual(result[()], 1.0, places=6)
AssertionError: np.float64(3.8307390766633826) != 1.0 within 6 places (np.float64(2.8307390766633826) difference)

----------------------------------------------------------------------
Ran 7 tests in 7.474s

FAILED (failures=1)


In [8]:
# @title B.2 Performance Tests for the Implementation
import numpy as np
import time
import tracemalloc
from pprint import pprint

np.random.seed(42)
def rand_multivector():
    return sum(np.random.randn() * blade/10 for blade in layout.blades_list)

# Benchmark one function
def benchmark_func(func, n_runs=500):
    times = []
    tracemalloc.start()
    for _ in range(n_runs):
        u = rand_multivector()
        start = time.perf_counter()
        try:
            result = func(u)
        except ZeroDivisionError:
            continue
        end = time.perf_counter()
        times.append(end - start)
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return {
        "mean_time_ms"   : 1e3 * np.mean(times),
        "std_time_ms"    : 1e3 * np.std(times),
        "peak_memory_kb" : peak / 1024,
        "successful_runs": len(times),
        "total_runs"     : n_runs}

# Run benchmarks
results = {
    "det_hitzer_sangwine": benchmark_func(cl7_determinant_hitzersangwine),
    "det"       : benchmark_func(cl7_determinant),
    "det_opt"   : benchmark_func(cl7_determinant_opt),
    "inv"       : benchmark_func(cl7_inverse),
    "inv_opt"   : benchmark_func(cl7_inverse_opt),
}